In [1]:
import pandas as pd
import gffutils
import pybedtools
import re
import numpy as np

v19db_filename = '/projects/ps-yeolab/genomes/hg19/gencode/v19/gencode.v19.annotation.gtf.db'
v19db = gffutils.FeatureDB(v19db_filename)

folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'
csv_folder = '/home/obotvinnik/projects/singlecell_pnms/analysis/csvs_for_paper'

splicing_feature_data = pd.read_csv('{}/splicing_feature_data.csv'.format(folder))
splicing_feature_data.head()

/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:784: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())
/home/obotvinnik/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (11,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,event_id,criteria,criteria_additional,criteria_full,ensembl_id,exon1,exon1_length,exon2,exon2_length,exon3,...,isoform1_rnafold_interior_loop_percentage,isoform2_rnafold_interior_loop_percentage,isoform1_rnafold_minimum_free_energy,isoform2_rnafold_minimum_free_energy,isoform1_rnafold_multiloop_percentage,isoform2_rnafold_multiloop_percentage,isoform1_rnafold_stem_percentage,isoform2_rnafold_stem_percentage,isoform1_rnafold_three_prime_percentage,isoform2_rnafold_three_prime_percentage
0,exon:chr10:100190328-100190427:-@exon:chr10:10...,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,99,exon:chr10:100189330-100189399:-,...,0.205882,0.223048,-47.1,-95.9,0.105882,0.052045,0.505882,0.572491,0.017647,0.011152
1,exon:chr10:100193697-100193848:-@exon:chr10:10...,only one,NaN,only one,ENSG00000107521,exon:chr10:100193697-100193848:-,152,exon:chr10:100190888-100191048:-,161,exon:chr10:100190328-100190427:-,...,0.095238,0.096852,-108.7,-183.1,0.079365,0.111380,0.650794,0.658596,0.023810,0.014528
2,exon:chr10:100195392-100195529:-@exon:chr10:10...,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100195392-100195529:-,138,exon:chr10:100195029-100195171:-,143,exon:chr10:100193697-100193848:-,...,0.182759,0.154734,-116.1,-168.8,0.013793,0.078522,0.675862,0.665127,0.006897,0.004619
3,exon:chr10:101165513-101165617:-@exon:chr10:10...,only one,NaN,only one,ENSG00000120053,exon:chr10:101165513-101165617:-,105,exon:chr10:101163481-101163631:-,151,exon:chr10:101163226-101163391:-,...,0.092251,0.191943,-79.6,-131.2,0.103321,0.063981,0.605166,0.630332,0.011070,0.002370
4,exon:chr10:101419263-101419345:+@exon:chr10:10...,only one,NaN,only one,ENSG00000198018,exon:chr10:101419263-101419345:+,83,exon:chr10:101419619-101419721:+,103,exon:chr10:101421203-101421385:+,...,0.105263,0.073171,-86.2,-122.2,0.142857,0.135501,0.609023,0.563686,0.041353,0.029810


In [3]:
junction_exons = pd.read_csv('{}/junction_exon_triples.csv'.format(folder))
print junction_exons.shape
junction_exons.head()

(844056, 3)


,exon,direction,junction
0,exon:chr1:29321-29370:-,downstream,chr1:18367-29320:-
1,exon:chr1:29321-29370:-,downstream,chr1:24892-29320:-
2,exon:chr1:29321-29370:-,downstream,chr1:27186-29320:-
3,exon:chr1:29321-29370:-,downstream,chr1:18380-29320:-
4,exon:chr1:29321-29370:-,downstream,chr1:18370-29320:-


In [2]:
folder = '/home/obotvinnik/projects/singlecell_pnms/analysis/csvs_for_paper'
se_events = pd.read_csv('{}/se_events_with_stressed.csv'.format(folder))
se_events.head()

,exon1,exon2,exon3,junction12,junction23,junction13,event_id
0,exon:chr10:51732523-51732902:+,exon:chr10:51740949-51741041:+,exon:chr10:51751436-51751504:+,chr10:51732903-51740948:+,chr10:51741042-51751435:+,chr10:51732903-51751435:+,exon:chr10:51732523-51732902:+@exon:chr10:5174...
1,exon:chr1:32669480-32669646:+,exon:chr1:32669787-32669980:+,exon:chr1:32670199-32670988:+,chr1:32669647-32669786:+,chr1:32669981-32670198:+,chr1:32669647-32670198:+,exon:chr1:32669480-32669646:+@exon:chr1:326697...
2,chr11:67374548-67375870:+,chr11:67375950-67376030:+,chr11:67376194-67376922:+,exon:chr11:67375871-67375949:+,exon:chr11:67376031-67376193:+,exon:chr11:67375871-67376193:+,chr11:67374548-67375870:+@chr11:67375950-67376...
3,exon:chr11:18425241-18425358:+,exon:chr11:18426996-18427119:+,exon:chr11:18428664-18429394:+,chr11:18425359-18426995:+,chr11:18427120-18428663:+,chr11:18425359-18428663:+,exon:chr11:18425241-18425358:+@exon:chr11:1842...
4,exon:chr10:99213311-99213420:+,exon:chr10:99213556-99213603:+,exon:chr10:99214471-99214556:+,chr10:99213421-99213555:+,chr10:99213604-99214470:+,chr10:99213421-99214470:+,exon:chr10:99213311-99213420:+@exon:chr10:9921...


In [4]:
mxe_events = pd.read_csv('{}/mxe_events_with_stressed.csv'.format(folder))
mxe_events.head()

,exon1,exon2,exon3,exon4,junction13,junction34,junction12,junction24,event_id
0,exon:chr8:144663224-144663324:-,exon:chr8:144668389-144668460:-,exon:chr8:144663456-144663498:-,exon:chr8:144668899-144669019:-,chr8:144663325-144663455:-,chr8:144663499-144668898:-,chr8:144663325-144668388:-,chr8:144668461-144668898:-,exon:chr8:144663224-144663324:-@exon:chr8:1446...
1,exon:chr19:8376168-8376479:-,exon:chr19:8385741-8385790:-,exon:chr19:8381380-8381529:-,exon:chr19:8386192-8386259:-,chr19:8376480-8381379:-,chr19:8381530-8386191:-,chr19:8376480-8385740:-,chr19:8385791-8386191:-,exon:chr19:8376168-8376479:-@exon:chr19:838574...
2,exon:chr17:33307578-33307641:+,exon:chr17:33310021-33310571:+,exon:chr17:33313007-33313150:+,exon:chr17:33316485-33316899:+,chr17:33307642-33313006:+,chr17:33313151-33316484:+,chr17:33307642-33310020:+,chr17:33310572-33316484:+,exon:chr17:33307578-33307641:+@exon:chr17:3331...
3,exon:chr14:31091525-31091605:+,exon:chr14:31097415-31097485:+,exon:chr14:31099683-31099771:+,exon:chr14:31103153-31103243:+,chr14:31091606-31099682:+,chr14:31099772-31103152:+,chr14:31091606-31097414:+,chr14:31097486-31103152:+,exon:chr14:31091525-31091605:+@exon:chr14:3109...
4,exon:chr1:247002400-247004300:-,exon:chr1:247007097-247007230:-,exon:chr1:247005996-247006078:-,exon:chr1:247012917-247013151:-,chr1:247004301-247005995:-,chr1:247006079-247012916:-,chr1:247004301-247007096:-,chr1:247007231-247012916:-,exon:chr1:247002400-247004300:-@exon:chr1:2470...


In [7]:
exon2s = pd.concat([se_events[['exon2', 'event_id']], mxe_events[['exon2', 'event_id']]], ignore_index=True)
print exon2s.shape
exon2s.head()

(136429, 2)


,exon2,event_id
0,exon:chr10:51740949-51741041:+,exon:chr10:51732523-51732902:+@exon:chr10:5174...
1,exon:chr1:32669787-32669980:+,exon:chr1:32669480-32669646:+@exon:chr1:326697...
2,chr11:67375950-67376030:+,chr11:67374548-67375870:+@chr11:67375950-67376...
3,exon:chr11:18426996-18427119:+,exon:chr11:18425241-18425358:+@exon:chr11:1842...
4,exon:chr10:99213556-99213603:+,exon:chr10:99213311-99213420:+@exon:chr10:9921...


## Write Exon2 from SE and MXE to bed file

In [3]:
from outrigger.region import Region

splicing_feature_data['exon2_region'] = splicing_feature_data.exon2.map(Region)

exon2_bed = pd.DataFrame.from_records(splicing_feature_data.exon2_region.map(
    lambda x: pd.Series(dict(chrom=x.chrom, start=x._start, stop=x._stop, strand=x.strand, score=1000))))

exon2_bed['name'] = splicing_feature_data.event_id
exon2_bed = exon2_bed.reindex(columns=['chrom', 'start', 'stop', 'name', 'score', 'strand'])

print exon2_bed.shape
exon2_bed.head()

(33693, 6)


,chrom,start,stop,name,score,strand
0,chr10,100189548,100189646,exon:chr10:100190328-100190427:-@exon:chr10:10...,1000,-
1,chr10,100190888,100191048,exon:chr10:100193697-100193848:-@exon:chr10:10...,1000,-
2,chr10,100195029,100195171,exon:chr10:100195392-100195529:-@exon:chr10:10...,1000,-
3,chr10,101163481,101163631,exon:chr10:101165513-101165617:-@exon:chr10:10...,1000,-
4,chr10,101419619,101419721,exon:chr10:101419263-101419345:+@exon:chr10:10...,1000,+


In [4]:
exon2_bed.to_csv('{}/exon2.bed'.format(folder), index=False, header=False, sep='\t')

## Get internal, constitutive exons

### Internal: have both downstream and upstream junctions

In [8]:
exon_direction = junction_exons.groupby(['exon', 'direction']).size()
exon_direction.head()
# constitutive = constitutive[constitutive == 2]

exon                              direction 
exon:chr10:100003848-100004106:+  downstream    1
exon:chr10:100003848-100004321:+  downstream    1
exon:chr10:100003848-100004651:+  downstream    1
exon:chr10:100003848-100004654:+  downstream    1
exon:chr10:100007447-100008748:-  upstream      1
dtype: int64

Group on the number of "direction" items they have. Must have two to be internal

In [9]:
internal_exons = exon_direction.groupby(level=0).size()
internal_exons = internal_exons[internal_exons == 2]
internal_exons.head()

exon
exon:chr10:100010822-100010933:-    2
exon:chr10:100144704-100144824:-    2
exon:chr10:100146958-100147064:-    2
exon:chr10:100150355-100150511:-    2
exon:chr10:100155148-100155209:-    2
dtype: int64

In [13]:
exon_juction_direction_internal = junction_exons.loc[junction_exons.exon.isin(internal_exons.index)]
print exon_juction_direction_internal.shape
exon_juction_direction_internal.head()

(448078, 3)


,exon,direction,junction
5,exon:chr1:24738-24891:-,upstream,chr1:24892-29320:-
6,exon:chr1:24738-24891:-,upstream,chr1:24892-29533:-
7,exon:chr1:24738-24891:-,downstream,chr1:18062-24737:-
8,exon:chr1:24738-24891:-,downstream,chr1:18380-24737:-
9,exon:chr1:24738-24891:-,downstream,chr1:18555-24737:-


### Remove all exons that appear as exon2 in any alternative annotations, or exon3 for MXE

In [15]:
alt_exons = pd.concat([se_events['exon2'], mxe_events['exon2'], mxe_events['exon3']], ignore_index=True)
print alt_exons.shape
alt_exons = alt_exons.drop_duplicates()
print alt_exons.shape

in_alt_exons = exon_juction_direction_internal.exon.isin(alt_exons.values)

constitutive = exon_juction_direction_internal.loc[(~in_alt_exons)]
print constitutive.shape
constitutive.head()

(166344,)
(22847,)
(360186, 3)


,exon,direction,junction
18,exon:chr1:17915-18061:-,upstream,chr1:18062-18267:-
19,exon:chr1:17915-18061:-,upstream,chr1:18062-24737:-
20,exon:chr1:17915-18061:-,downstream,chr1:17743-17914:-
21,exon:chr1:17915-18061:-,downstream,chr1:17056-17914:-
24,exon:chr1:16854-17055:-,upstream,chr1:17056-17232:-


In [16]:
from poshsplice.region import Region
constitutive_regions = pd.Series(constitutive.exon.unique()).map(Region)
print constitutive_regions.shape
constitutive_regions.head()

(128677,)


0    exon:chr1:17915-18061:-
1    exon:chr1:16854-17055:-
2    exon:chr1:16607-16765:-
3    exon:chr1:14970-15038:-
4    exon:chr1:17606-17742:-
dtype: object

In [18]:
constitutive_junctions = constitutive.junction.unique()
len(constitutive_junctions)

231088

In [20]:
records = constitutive_regions.map(
    lambda x: pd.Series(dict(chrom=x.chrom, start=x._start, stop=x._stop, strand=x.strand, score=1000, name=x.name)))
constitutive_bed = pd.DataFrame.from_records(records)
constitutive_bed = constitutive_bed.reindex(columns=['chrom', 'start', 'stop', 'name', 'score', 'strand'])
print constitutive_bed.shape
constitutive_bed.head()

(128677, 6)


,chrom,start,stop,name,score,strand
0,chr1,17915,18061,exon:chr1:17915-18061:-,1000,-
1,chr1,16854,17055,exon:chr1:16854-17055:-,1000,-
2,chr1,16607,16765,exon:chr1:16607-16765:-,1000,-
3,chr1,14970,15038,exon:chr1:14970-15038:-,1000,-
4,chr1,17606,17742,exon:chr1:17606-17742:-,1000,-


In [21]:
constitutive_bed = constitutive_bed.drop_duplicatescates()
constitutive_bed.shape

(128677, 6)

In [23]:
constitutive_bed.to_csv('{}/constitutive_exons.bed'.format(folder), sep='\t', index=False, header=False)

## Submit a job to calculate conservation

In [24]:
import os
import glob

from gscripts.qtools import Submitter


folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'

alt_exons_bedfile = '{}/exon2.bed'.format(folder)
constitutive_bedfile = '{}/constitutive_exons.bed'.format(folder)

bedfiles = alt_exons_bedfile, constitutive_bedfile

commands = []

bw = '/projects/ps-yeolab/genomes/hg19/hg19_phastcons_placental_mammal.bw'

for bedfile in bedfiles:
    basename = os.path.basename(bedfile)
    
    prefix = basename.split('.bed')[0]
    
    prefix += '_phastcons_placental_mammal'
    bedout = '{}/{}'.format(folder, prefix + '.bed')
    outtab = '{}/{}'.format(folder, prefix + '.txt')
    command = 'bigWigAverageOverBed {} {} {} -bedOut={}'.format(bw, bedfile, outtab, bedout)
    print command
    commands.append(command)

jobname = 'exonbody_conservation'
Submitter(commands, jobname, array=True, walltime='2:00:00', write_and_submit=True,
          err_filename='{}/{}.err'.format(folder, jobname),
          out_filename='{}/{}.out'.format(folder, jobname))

running 2 tasks as an array-job.
job ID: 3800347


bigWigAverageOverBed /projects/ps-yeolab/genomes/hg19/hg19_phastcons_placental_mammal.bw /projects/ps-yeolab/obotvinnik/singlecell_pnms/exon2.bed /projects/ps-yeolab/obotvinnik/singlecell_pnms/exon2_phastcons_placental_mammal.txt -bedOut=/projects/ps-yeolab/obotvinnik/singlecell_pnms/exon2_phastcons_placental_mammal.bed
bigWigAverageOverBed /projects/ps-yeolab/genomes/hg19/hg19_phastcons_placental_mammal.bw /projects/ps-yeolab/obotvinnik/singlecell_pnms/constitutive_exons.bed /projects/ps-yeolab/obotvinnik/singlecell_pnms/constitutive_exons_phastcons_placental_mammal.txt -bedOut=/projects/ps-yeolab/obotvinnik/singlecell_pnms/constitutive_exons_phastcons_placental_mammal.bed


# Don't do below -- too much work and code crashes and not worth it

### Filter constitutive junctions for usage

Check that the junctions for these exons are seen in 20+ single cells per celltype

In [11]:
csv_folder = '/home/obotvinnik/projects/singlecell_pnms/analysis/csvs_for_paper'
splicing =  pd.read_csv('{}/splicing.csv'.format(csv_folder), index_col=0)
print splicing.shape
splicing.head()

(233, 25836)


,exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-,exon:chr10:100193697-100193848:-@exon:chr10:100190888-100191048:-@exon:chr10:100190328-100190427:-,exon:chr10:100195392-100195529:-@exon:chr10:100195029-100195171:-@exon:chr10:100193697-100193848:-,exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,exon:chr10:101419263-101419345:+@exon:chr10:101419619-101419721:+@exon:chr10:101421203-101421385:+,exon:chr10:101421203-101421385:+@exon:chr10:101439018-101439223:+@exon:chr10:101439482-101439632:+,exon:chr10:101480744-101480825:-@exon:chr10:101478103-101478257:-@exon:chr10:101476105-101476218:-,exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+,exon:chr10:101507014-101507147:+@exon:chr10:101514286-101514391:+@exon:chr10:101515382-101515891:+,exon:chr10:101964263-101964414:-@exon:chr10:101961845-101961906:-@exon:chr10:101960428-101960537:-,...,exon:chrY:15471647-15471866:-@exon:chrY:15470968-15471102:-@exon:chrY:15470344-15470433:-@exon:chrY:15469757-15469849:-,exon:chrY:16634632-16634821:+@exon:chrY:16733889-16734471:+@exon:chrY:16831339-16831398:+@exon:chrY:16834997-16835149:+,exon:chrY:21237827-21237882:-@exon:chrY:21230610-21230684:-@exon:chrY:21207128-21207177:-@exon:chrY:21206428-21206581:-,exon:chrY:21749096-21749393:+@exon:chrY:21750256-21751498:+@exon:chrY:21753666-21753845:+@exon:chrY:21755285-21755479:+,exon:chrY:21749368-21749393:+@exon:chrY:21751407-21751498:+@exon:chrY:21753666-21753845:+@exon:chrY:21755285-21755479:+,exon:chrY:21903621-21903743:-@exon:chrY:21903204-21903374:-@exon:chrY:21901414-21901548:-@exon:chrY:21897507-21897636:-,exon:chrY:2709527-2709668:+@exon:chrY:2710206-2710283:+@exon:chrY:2712118-2712298:+@exon:chrY:2713687-2713784:+,exon:chrY:2710206-2710283:+@exon:chrY:2712118-2712298:+@exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+,exon:chrY:2712118-2712298:+@exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+@exon:chrY:2733129-2733286:+,exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+@exon:chrY:2733129-2733286:+@exon:chrY:2734834-2735309:+
CVN_01,NaN,NaN,NaN,0,NaN,0,NaN,0,NaN,NaN,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CVN_02,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CVN_03,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CVN_04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CVN_05,NaN,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
folder = '/home/obotvinnik/projects/singlecell_pnms/analysis/csvs_for_paper'
sj = pd.read_csv('{}/sj_raw.csv'.format(folder))
    
# print sj.shape
# sj = sj.drop_duplicates(['sample_id', 'intron_location'])
# print sj.shape
    
# sj = sj.set_index(['intron_location', 'sample_id'])
# sj = sj.sort_index()
sj.head()

,chrom,intron_start,intron_stop,strand,intron_motif,annotated,unique_junction_reads,multimap_junction_reads,max_overhang,intron_location,sample_id
0,chr1,1309283,1309379,-,GT/AG,True,37,0,49,chr1:1309283-1309379:-,CVN_09
1,chr1,1309826,1310084,-,GT/AG,True,43,0,49,chr1:1309826-1310084:-,CVN_09
2,chr1,1310171,1310533,-,GT/AG,True,27,0,47,chr1:1310171-1310533:-,CVN_09
3,chr1,1337637,1341188,-,GT/AG,True,409,0,50,chr1:1337637-1341188:-,CVN_09
4,chr1,1341267,1342288,-,GT/AG,True,337,0,50,chr1:1341267-1342288:-,CVN_09


### Get just junctions associated with constitutive exons

In [13]:
sj_constitutive = sj.loc[sj['intron_location'].isin(constitutive_junctions)]
print sj_constitutive.shape
sj_constitutive.head()

(3693174, 11)


,chrom,intron_start,intron_stop,strand,intron_motif,annotated,unique_junction_reads,multimap_junction_reads,max_overhang,intron_location,sample_id
0,chr1,1309283,1309379,-,GT/AG,True,37,0,49,chr1:1309283-1309379:-,CVN_09
1,chr1,1309826,1310084,-,GT/AG,True,43,0,49,chr1:1309826-1310084:-,CVN_09
3,chr1,1337637,1341188,-,GT/AG,True,409,0,50,chr1:1337637-1341188:-,CVN_09
4,chr1,1341267,1342288,-,GT/AG,True,337,0,50,chr1:1341267-1342288:-,CVN_09
5,chr1,1342400,1342510,-,GT/AG,True,307,0,50,chr1:1342400-1342510:-,CVN_09


In [14]:
metadata = pd.read_csv('{}/metadata.csv'.format(csv_folder), index_col=0)
metadata.head()

,single,pooled,phenotype,neuron,progenitor,stressed,craig_venter,differentiation_batch,split
CVN_01,True,False,NPC,False,True,False,True,NaN,NaN
CVN_02,True,False,NPC,False,True,False,True,NaN,NaN
CVN_03,True,False,NPC,False,True,False,True,NaN,NaN
CVN_04,True,False,NPC,False,True,False,True,NaN,NaN
CVN_05,True,False,NPC,False,True,False,True,NaN,NaN


In [15]:
phenotype = metadata.phenotype[metadata.single]

### Call an exon constitutive if its junction start/stop is detected in at least 20 cells per celltype

In [16]:
sj_constitutive = sj_constitutive.set_index('sample_id')
sj_constitutive.head()

,chrom,intron_start,intron_stop,strand,intron_motif,annotated,unique_junction_reads,multimap_junction_reads,max_overhang,intron_location
sample_id,,,,,,,,,,
CVN_09,chr1,1309283,1309379,-,GT/AG,True,37,0,49,chr1:1309283-1309379:-
CVN_09,chr1,1309826,1310084,-,GT/AG,True,43,0,49,chr1:1309826-1310084:-
CVN_09,chr1,1337637,1341188,-,GT/AG,True,409,0,50,chr1:1337637-1341188:-
CVN_09,chr1,1341267,1342288,-,GT/AG,True,337,0,50,chr1:1341267-1342288:-
CVN_09,chr1,1342400,1342510,-,GT/AG,True,307,0,50,chr1:1342400-1342510:-


In [17]:
for p, phenotype_df in sj_constitutive.groupby(phenotype):
    for i, (intron, intron_df) in zip(range(50), phenotype_df.groupby(['chrom', 'intron_stop', 'strand'])):
        n_unique_samples = len(intron_df.index.unique())
        if n_unique_samples >= 20:
            print intron, n_unique_samples, intron_df.shape

('chr1', 155766.0, '-') 27 (35, 10)
('chr1', 164262.0, '-') 26 (26, 10)
('chr1', 880436.0, '-') 22 (22, 10)
('chr1', 880897.0, '-') 29 (29, 10)
('chr1', 881552.0, '-') 30 (30, 10)
('chr1', 881781.0, '-') 30 (31, 10)
('chr1', 883510.0, '-') 30 (30, 10)
('chr1', 883869.0, '-') 29 (29, 10)
('chr1', 887379.0, '-') 21 (21, 10)
('chr1', 887791.0, '-') 24 (24, 10)
('chr1', 888554.0, '-') 26 (26, 10)
('chr1', 889161.0, '-') 26 (26, 10)
('chr1', 889383.0, '-') 24 (24, 10)
('chr1', 891302.0, '-') 25 (25, 10)
('chr1', 891474.0, '-') 25 (25, 10)
('chr1', 892273.0, '-') 22 (22, 10)
('chr1', 892478.0, '-') 22 (22, 10)
('chr1', 894594.0, '-') 27 (27, 10)


In [18]:
intron_df

,chrom,intron_start,intron_stop,strand,intron_motif,annotated,unique_junction_reads,multimap_junction_reads,max_overhang,intron_location
sample_id,,,,,,,,,,
P4_12,chr1,1164327,1167271,-,GT/AG,True,18,0,49,chr1:1164327-1167271:-
P7_10,chr1,1164327,1167271,-,GT/AG,True,32,0,50,chr1:1164327-1167271:-
P8_10,chr1,1164327,1167271,-,GT/AG,True,10,0,49,chr1:1164327-1167271:-
P7_09,chr1,1164327,1167271,-,GT/AG,True,32,0,50,chr1:1164327-1167271:-
P1_09,chr1,1164327,1167271,-,GT/AG,True,15,0,45,chr1:1164327-1167271:-
P2_10,chr1,1164327,1167271,-,GT/AG,True,13,0,46,chr1:1164327-1167271:-
P2_07,chr1,1164327,1167271,-,GT/AG,True,47,0,43,chr1:1164327-1167271:-
P8_05,chr1,1164327,1167271,-,GT/AG,True,67,0,49,chr1:1164327-1167271:-


In [23]:
s = sj_constitutive.groupby(['chrom', 'intron_stop', 'strand']).size()
len(s)

148708

In [30]:
sj_constitutive.shape

(3693174, 10)

In [29]:
chrom = 'chrom'
start = 'intron_start'
stop = 'intron_stop'
strand = 'strand'

sj_constitutive_stop = sj_constitutive.groupby([chrom, stop, strand]).filter(lambda x: len(x) >= 20)
sj_constitutive_stop.shape

(3147848, 10)

In [25]:
(s>=20).sum()

48075

In [64]:
for i, (group, df) in zip(range(20), sj_constitutive.groupby(['chrom', 'intron_stop', 'strand'])):
    s = df.groupby(phenotype).size()
    if (s >= 20).any():
        print s[s >= 20]
        break
    

phenotype
iPSC    35
dtype: int64


In [71]:
grouped = df.groupby(phenotype)
s = grouped.size()

pd.concat([grouped.get_group(g) for g in s[s >= 20].index])

,chrom,intron_start,intron_stop,strand,intron_motif,annotated,unique_junction_reads,multimap_junction_reads,max_overhang,intron_location
sample_id,,,,,,,,,,
P4_06,chr1,146510,155766,-,GT/AG,True,12,23,49,chr1:146510-155766:-
P2_03,chr1,146510,155766,-,GT/AG,True,10,5,34,chr1:146510-155766:-
P1_01,chr1,146510,155766,-,GT/AG,True,19,22,44,chr1:146510-155766:-
P8_10,chr1,146510,155766,-,GT/AG,True,60,103,49,chr1:146510-155766:-
P7_02,chr1,146510,155766,-,GT/AG,True,25,16,49,chr1:146510-155766:-
P9_01,chr1,146510,155766,-,GT/AG,True,129,159,50,chr1:146510-155766:-
P9_01,chr1,153544,155766,-,GT/AG,False,58,60,49,chr1:153544-155766:-
P9_02,chr1,146510,155766,-,GT/AG,True,118,165,49,chr1:146510-155766:-
P7_11,chr1,153544,155766,-,GT/AG,False,20,33,49,chr1:153544-155766:-


In [60]:
s

phenotype
MN      2
NPC     1
iPSC    1
dtype: int64

In [80]:
def filter_group_size(df, groupby, size):
    grouped = df.groupby(phenotype)
    s = grouped.size()
    ind = s >= 20
#     if (ind).any():
#         return pd.concat([grouped.get_group(g) for g in s[ind].index])
#     else:
#         return pd.DataFrame()
    return s[ind]

%time constitutive_stops = sj_constitutive.groupby(['chrom', 'intron_stop', 'strand', 'intron_location']).apply(filter_group_size, groupby=phenotype, size=20)

CPU times: user 5min 21s, sys: 304 ms, total: 5min 22s
Wall time: 5min 22s


In [82]:
constitutive_stops_phenotype = constitutive_stops.reset_index(level=-1)
constitutive_stops_phenotype.phenotype.head()

chrom  intron_stop  strand  intron_location     
chr1   155766       -       chr1:146510-155766:-    iPSC
       164262       -       chr1:155832-164262:-    iPSC
       880436       -       chr1:880181-880436:-    iPSC
       880897       -       chr1:880527-880897:-    iPSC
       881552       -       chr1:881034-881552:-    iPSC
Name: phenotype, dtype: object

In [83]:
%%time
constitutive_starts = sj_constitutive.groupby(['chrom', 'intron_start', 'strand', 'intron_location']).apply(
    filter_group_size, groupby=phenotype, size=20)

CPU times: user 5min 15s, sys: 788 ms, total: 5min 16s
Wall time: 5min 16s


In [85]:
constitutive_starts.head()

chrom  intron_start  strand  intron_location       phenotype
chr1   146510        -       chr1:146510-155766:-  iPSC         23
       155832        -       chr1:155832-164262:-  iPSC         25
       880181        -       chr1:880181-880436:-  iPSC         22
       880527        -       chr1:880527-880897:-  iPSC         29
       881034        -       chr1:881034-881552:-  iPSC         30
dtype: int64

In [86]:
constitutive_stops_phenotype = constitutive_stops.reset_index(level=-1)
constitutive_stops_phenotype.phenotype.head()

chrom  intron_stop  strand  intron_location     
chr1   155766       -       chr1:146510-155766:-    iPSC
       164262       -       chr1:155832-164262:-    iPSC
       880436       -       chr1:880181-880436:-    iPSC
       880897       -       chr1:880527-880897:-    iPSC
       881552       -       chr1:881034-881552:-    iPSC
Name: phenotype, dtype: object

In [53]:
constitutive_starts_phenotype = constitutive_starts.reset_index(level=-1)
constitutive_starts_phenotype.phenotype.head()

chrom  intron_start  strand
chr1   146510        -         iPSC
       155832        -         iPSC
       880181        -         iPSC
       880527        -         iPSC
       881034        -         iPSC
Name: phenotype, dtype: object

In [49]:
constitutive['junction_region'] = constitutive.junction_location.apply(Region)
constitutive['intron_start'] = constitutive.junction_region.apply(lambda x: x._start)
constitutive['intron_stop'] = constitutive.junction_region.apply(lambda x: x._stop)
constitutive['chrom'] = constitutive.junction_region.apply(lambda x: x.chrom)
constitutive['strand'] = constitutive.junction_region.apply(lambda x: x.strand)
constitutive.head()

/home/obotvinnik/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/obotvinnik/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/obotvinnik/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

,junction_location,exon,direction,junction_region,intron_start,intron_stop,chrom,strand
0,chr10:100008749-100010821:-,exon:chr10:100010822-100010933:-,upstream,chr10:100008749-100010821:-,100008749,100010821,chr10,-
2,chr10:100143626-100144703:-,exon:chr10:100144704-100144824:-,upstream,chr10:100143626-100144703:-,100143626,100144703,chr10,-
3,chr10:100144825-100146957:-,exon:chr10:100146958-100147064:-,upstream,chr10:100144825-100146957:-,100144825,100146957,chr10,-
4,chr10:100147065-100155147:-,exon:chr10:100155148-100155209:-,upstream,chr10:100147065-100155147:-,100147065,100155147,chr10,-
6,chr10:100155210-100157101:-,exon:chr10:100157102-100157255:-,upstream,chr10:100155210-100157101:-,100155210,100157101,chr10,-


In [50]:
constitutive_index = constitutive.set_index(['chrom', 'intron_start', 'strand'])
constitutive_index.head()

junction_location  \
chrom intron_start strand                                
chr10 100008749    -       chr10:100008749-100010821:-   
      100143626    -       chr10:100143626-100144703:-   
      100144825    -       chr10:100144825-100146957:-   
      100147065    -       chr10:100147065-100155147:-   
      100155210    -       chr10:100155210-100157101:-   

                                                       exon direction  \
chrom intron_start strand                                               
chr10 100008749    -       exon:chr10:100010822-100010933:-  upstream   
      100143626    -       exon:chr10:100144704-100144824:-  upstream   
      100144825    -       exon:chr10:100146958-100147064:-  upstream   
      100147065    -       exon:chr10:100155148-100155209:-  upstream   
      100155210    -       exon:chr10:100157102-100157255:-  upstream   

                                       junction_region  intron_stop  
chrom intron_start strand                                            
chr10 100008749    -       chr10:100008749-100010821:-    100010821  
      100143626    -       chr10:100143626-100144703:-    100144703  
      100144825    -       chr10:100144825-100146957:-    100146957  
      100147065    -       chr10:100147065-100155147:-    100155147  
      100155210    -       chr10:100155210-100157101:-    100157101

In [57]:
for i, (exon, df) in zip(range(5), constitutive.groupby('exon')):
    print df.head()

             junction_location                              exon   direction  \
0  chr10:100008749-100010821:-  exon:chr10:100010822-100010933:-    upstream   
1  chr10:100010934-100011322:-  exon:chr10:100010822-100010933:-  downstream   

               junction_region  intron_start  intron_stop  chrom strand  
0  chr10:100008749-100010821:-     100008749    100010821  chr10      -  
1  chr10:100010934-100011322:-     100010934    100011322  chr10      -  
             junction_location                              exon   direction  \
2  chr10:100143626-100144703:-  exon:chr10:100144704-100144824:-    upstream   
3  chr10:100144825-100146957:-  exon:chr10:100144704-100144824:-  downstream   

               junction_region  intron_start  intron_stop  chrom strand  
2  chr10:100143626-100144703:-     100143626    100144703  chr10      -  
3  chr10:100144825-100146957:-     100144825    100146957  chr10      -  
             junction_location                              exon   directi

In [ ]:
# sj_constitutive = sj.loc[idx[constitutive_junctions, :], :]
# print sj_constitutive.shape

In [1]:
# sj_constitutive.head()

NameError: name 'sj_constitutive' is not defined

In [ ]:
sj.reset_index().join

In [36]:
psi20_all = psi3_psi5.loc[:, (psi3_psi5.groupby(phenotype).count() >= 20).all()]
psi20_all.head()

,chr10:101157444-101162336:-|5p,chr10:101162480-101163225:-|5p,chr10:101163392-101163480:-|5p,chr10:101163392-101165512:-|5p,chr10:101163632-101165512:-|5p,chr10:101166007-101190204:-|5p,chr10:101437763-101476104:-|5p,chr10:101456299-101476104:-|5p,chr10:101473237-101476104:-|5p,chr10:101474476-101476104:-|5p,...,chrY:2713785-2734833:+|3p,chrY:2714508-2722640:+|3p,chrY:2721733-2722640:+|3p,chrY:2722746-2734833:+|3p,chrY:2722813-2733128:+|3p,chrY:2722813-2734833:+|3p,chrY:2722813-2789783:+|3p,chrY:2722813-2796904:+|3p,chrY:2733287-2734833:+|3p,chrY:2733362-2734833:+|3p
sample_id,,,,,,,,,,,,,,,,,,,,,
CVN_01,1,1,1,0,1,0,NaN,NaN,NaN,NaN,...,0,0,0,0,1,0,0,0,1,0
CVN_02,1,1,1,0,1,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CVN_03,1,1,1,0,1,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CVN_04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,1,0,0,0,1,0
CVN_05,1,1,1,0,1,0,0,1,0,1,...,0,0,0,0,1,0,0,0,1,0


Get Junctions whose psi score is equal to 1 in all celltypes

In [39]:
psi_is_1 = psi20_all.groupby(phenotype).apply(lambda x: (x == 1)[x.notnull()].all(axis=0))
psi_is_1.head()

,chr10:101157444-101162336:-|5p,chr10:101162480-101163225:-|5p,chr10:101163392-101163480:-|5p,chr10:101163392-101165512:-|5p,chr10:101163632-101165512:-|5p,chr10:101166007-101190204:-|5p,chr10:101437763-101476104:-|5p,chr10:101456299-101476104:-|5p,chr10:101473237-101476104:-|5p,chr10:101474476-101476104:-|5p,...,chrY:2713785-2734833:+|3p,chrY:2714508-2722640:+|3p,chrY:2721733-2722640:+|3p,chrY:2722746-2734833:+|3p,chrY:2722813-2733128:+|3p,chrY:2722813-2734833:+|3p,chrY:2722813-2789783:+|3p,chrY:2722813-2796904:+|3p,chrY:2733287-2734833:+|3p,chrY:2733362-2734833:+|3p
phenotype,,,,,,,,,,,,,,,,,,,,,
MN,True,True,False,False,True,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
NPC,True,True,True,False,True,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,False
iPSC,True,True,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [40]:
psi20_is_1_all = psi20_all.loc[:, psi_is_1.all()]
psi20_is_1_all.head()

,chr10:101157444-101162336:-|5p,chr10:101162480-101163225:-|5p,chr10:101163632-101165512:-|5p,chr10:101480826-101483712:-|5p,chr10:101483881-101486724:-|5p,chr10:101487321-101489309:-|5p,chr10:101489492-101491716:-|5p,chr10:102035251-102039880:-|5p,chr10:102107289-102107820:+|5p,chr10:102286312-102286731:-|5p,...,chrX:77378872-77380370:+|3p,chrX:77380923-77381286:+|3p,chrX:80532669-80533829:+|3p,chrX:80533911-80552693:+|3p,chrX:99887566-99888401:-|3p,chrX:99890250-99890554:-|3p,chrY:1455653-1456171:-|3p,chrY:1456313-1458133:-|3p,chrY:14774638-14776570:+|3p,chrY:2710284-2712117:+|3p
sample_id,,,,,,,,,,,,,,,,,,,,,
CVN_01,1,1,1,NaN,NaN,NaN,NaN,1,NaN,1,...,1,1,1,1,1,1,1,1,1,1
CVN_02,1,1,1,NaN,NaN,NaN,NaN,NaN,1,1,...,1,1,1,1,1,1,1,1,NaN,NaN
CVN_03,1,1,1,NaN,NaN,NaN,NaN,1,1,1,...,NaN,NaN,1,1,NaN,NaN,1,1,1,NaN
CVN_04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,...,1,1,1,1,1,1,1,1,1,1
CVN_05,1,1,1,1,1,1,1,NaN,NaN,1,...,1,1,1,1,1,1,1,1,1,1


In [42]:
sj20_is_1_all = sj_metadata.loc[psi20_is_1_all.columns, :]
print sj20_is_1_all.shape
sj20_is_1_all.head()

(9313, 43)


,chrom,intron_start,intron_stop,strand,intron_motif,annotated,gencode_id,ensembl_id,gene_id,gene_name,...,gerstberger2014_rbp_target_trna,gerstberger2014_tf,rbp,ribosomal,ribosomal_subunit,synapse,transcription_factor,intron_length,exon_stop,exon_start
chr10:101157444-101162336:-|5p,chr10,101157444,101162336,-,GT/AG,True,ENSG00000120053.9,ENSG00000120053,ENSG00000120053.9,GOT1,...,False,False,False,False,False,False,False,4892,101157443,101162337
chr10:101162480-101163225:-|5p,chr10,101162480,101163225,-,GT/AG,True,ENSG00000120053.9,ENSG00000120053,ENSG00000120053.9,GOT1,...,False,False,False,False,False,False,False,745,101162479,101163226
chr10:101163632-101165512:-|5p,chr10,101163632,101165512,-,GT/AG,True,ENSG00000120053.9,ENSG00000120053,ENSG00000120053.9,GOT1,...,False,False,False,False,False,False,False,1880,101163631,101165513
chr10:101480826-101483712:-|5p,chr10,101480826,101483712,-,GT/AG,True,ENSG00000014919.8,ENSG00000014919,ENSG00000014919.8,COX15,...,False,False,False,False,False,False,False,2886,101480825,101483713
chr10:101483881-101486724:-|5p,chr10,101483881,101486724,-,GT/AG,True,ENSG00000014919.8,ENSG00000014919,ENSG00000014919.8,COX15,...,False,False,False,False,False,False,False,2843,101483880,101486725


In [43]:
records = constitutive_regions.map(
    lambda x: pd.Series(dict(chrom=x.chrom, start=x._start, stop=x._stop, strand=x.strand, score=1000, name=x.name)))
constitutive_bed = pd.DataFrame.from_records(records)
constitutive_bed = constitutive_bed.reindex(columns=['chrom', 'start', 'stop', 'name', 'score', 'strand'])
constitutive_bed.head()

,chrom,start,stop,name,score,strand
0,chr10,100010822,100010933,exon:chr10:100010822-100010933:-,1000,-
1,chr10,100144704,100144824,exon:chr10:100144704-100144824:-,1000,-
2,chr10,100146958,100147064,exon:chr10:100146958-100147064:-,1000,-
3,chr10,100155148,100155209,exon:chr10:100155148-100155209:-,1000,-
4,chr10,100157102,100157255,exon:chr10:100157102-100157255:-,1000,-


In [47]:
constitutive_exon_start.merge?

In [46]:
constitutive_exon_stop = constitutive_bed.merge(sj20_is_1_all, 
                                                right_on=['chrom', 'exon_stop', 'strand'], 
                                                left_on=['chrom', 'stop', 'strand'])
print 'constitutive_exon_stop', constitutive_exon_stop.shape
constitutive_exon_start = constitutive_bed.merge(sj20_is_1_all, 
                                                right_on=['chrom', 'exon_start', 'strand'], 
                                                left_on=['chrom', 'start', 'strand'])
print 'constitutive_exon_start', constitutive_exon_start.shape

constitutive_exon_stop (3501, 47)
constitutive_exon_start (3420, 47)


In [48]:
constitutive_exon_start.head()

,chrom,start,stop,name,score,strand,intron_start,intron_stop,intron_motif,annotated,...,gerstberger2014_rbp_target_trna,gerstberger2014_tf,rbp,ribosomal,ribosomal_subunit,synapse,transcription_factor,intron_length,exon_stop,exon_start
0,chr10,101162337,101162479,exon:chr10:101162337-101162479:-,1000,-,101157444,101162336,GT/AG,True,...,False,False,False,False,False,False,False,4892,101157443,101162337
1,chr10,101162337,101162479,exon:chr10:101162337-101162479:-,1000,-,101157444,101162336,GT/AG,True,...,False,False,False,False,False,False,False,4892,101157443,101162337
2,chr10,101483713,101483880,exon:chr10:101483713-101483880:-,1000,-,101480826,101483712,GT/AG,True,...,False,False,False,False,False,False,False,2886,101480825,101483713
3,chr10,101483713,101483880,exon:chr10:101483713-101483880:-,1000,-,101480826,101483712,GT/AG,True,...,False,False,False,False,False,False,False,2886,101480825,101483713
4,chr10,101486725,101486911,exon:chr10:101486725-101486911:-,1000,-,101483881,101486724,GT/AG,True,...,False,False,False,False,False,False,False,2843,101483880,101486725


In [50]:
constitutive_exons = constitutive_exon_start.merge(constitutive_exon_stop, left_on='name', right_on='name')
print constitutive_exons.shape
constitutive_exons.head()

(5229, 93)


,chrom_x,start_x,stop_x,name,score_x,strand_x,intron_start_x,intron_stop_x,intron_motif_x,annotated_x,...,gerstberger2014_rbp_target_trna_y,gerstberger2014_tf_y,rbp_y,ribosomal_y,ribosomal_subunit_y,synapse_y,transcription_factor_y,intron_length_y,exon_stop_y,exon_start_y
0,chr10,101162337,101162479,exon:chr10:101162337-101162479:-,1000,-,101157444,101162336,GT/AG,True,...,False,False,False,False,False,False,False,745,101162479,101163226
1,chr10,101162337,101162479,exon:chr10:101162337-101162479:-,1000,-,101157444,101162336,GT/AG,True,...,False,False,False,False,False,False,False,745,101162479,101163226
2,chr10,101162337,101162479,exon:chr10:101162337-101162479:-,1000,-,101157444,101162336,GT/AG,True,...,False,False,False,False,False,False,False,745,101162479,101163226
3,chr10,101162337,101162479,exon:chr10:101162337-101162479:-,1000,-,101157444,101162336,GT/AG,True,...,False,False,False,False,False,False,False,745,101162479,101163226
4,chr10,101483713,101483880,exon:chr10:101483713-101483880:-,1000,-,101480826,101483712,GT/AG,True,...,False,False,False,False,False,False,False,2843,101483880,101486725


In [52]:
bed = constitutive_exons[['chrom_x', 'start_x', 'stop_x', 'name', 'score_x', 'strand_x']]
bed = bed.drop_duplicates()
print bed.shape
bed.columns = bed.columns.map(lambda x: x.split('_')[0])
bed.head()

(1449, 6)


,chrom,start,stop,name,score,strand
0,chr10,101162337,101162479,exon:chr10:101162337-101162479:-,1000,-
4,chr10,101483713,101483880,exon:chr10:101483713-101483880:-,1000,-
8,chr10,101489310,101489491,exon:chr10:101489310-101489491:-,1000,-
12,chr10,104183414,104183474,exon:chr10:104183414-104183474:-,1000,-
16,chr10,104245366,104245490,exon:chr10:104245366-104245490:-,1000,-


In [58]:
bed.to_csv('{}/constitutive_exons.bed'.format(folder), index=False, header=False, sep='\t')

## Submit compute job to calculate exon conservation

In [28]:
import os
import glob

from gscripts.qtools import Submitter


folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'

alt_exons_bedfile = '{}/exon2.bed'.format(folder)
constitutive_bedfile = '{}/constitutive_exons.bed'.format(folder)

bedfiles = alt_exons_bedfile, constitutive_bedfile

commands = []

bw = '/projects/ps-yeolab/genomes/hg19/hg19_phastcons_placental_mammal.bw'

for bedfile in bedfiles:
    basename = os.path.basename(bedfile)
    
    prefix = basename.split('.bed')[0]
    
    prefix += '_phastcons_placental_mammal'
    bedout = '{}/{}'.format(folder, prefix + '.bed')
    outtab = '{}/{}'.format(folder, prefix + '.txt')
    command = 'bigWigAverageOverBed {} {} {} -bedOut={}'.format(bw, bedfile, outtab, bedout)
    print command
    commands.append(command)

jobname = 'exonbody_conservation'
Submitter(commands, jobname, array=True, walltime='2:00:00', write_and_submit=True,
          err_filename='{}/{}.err'.format(folder, jobname),
          out_filename='{}/{}.out'.format(folder, jobname))

running 2 tasks as an array-job.
job ID: 3614584


bigWigAverageOverBed /projects/ps-yeolab/genomes/hg19/hg19_phastcons_placental_mammal.bw /projects/ps-yeolab/obotvinnik/singlecell_pnms/exon2.bed /projects/ps-yeolab/obotvinnik/singlecell_pnms/exon2_phastcons_placental_mammal.txt -bedOut=/projects/ps-yeolab/obotvinnik/singlecell_pnms/exon2_phastcons_placental_mammal.bed
bigWigAverageOverBed /projects/ps-yeolab/genomes/hg19/hg19_phastcons_placental_mammal.bw /projects/ps-yeolab/obotvinnik/singlecell_pnms/constitutive_exons.bed /projects/ps-yeolab/obotvinnik/singlecell_pnms/constitutive_exons_phastcons_placental_mammal.txt -bedOut=/projects/ps-yeolab/obotvinnik/singlecell_pnms/constitutive_exons_phastcons_placental_mammal.bed
